In [56]:
import numpy as np
from scipy.optimize import linprog

Input:

12 9  
1 1 2 6 9  
1 0 2 7 10  
1 0 1 8 11  
1 0 4 5 9  
1 1 3 5 10  
1 2 3 4 11  
1 3 7 8 9  
1 4 6 8 10  
1 5 6 7 11  

In [36]:
n_cols, n_rows = input("Enter number of columns and rows: ").split()
n_cols = int(n_cols)
n_rows = int(n_rows)

cover_mat = np.zeros((n_rows, n_cols), dtype=np.int8)
row_weights = np.zeros(n_rows, dtype=int)

for i in range(n_rows):
    covered_columns = list(map(int, input().split()))
    row_weight = covered_columns[0]
    
    row_weights[i] = row_weight
    cover_mat[i, covered_columns[1:]] = True

Enter number of columns and rows: 12 9
1 1 2 6 9
1 0 2 7 10
1 0 1 8 11
1 0 4 5 9
1 1 3 5 10
1 2 3 4 11
1 3 7 8 9
1 4 6 8 10
1 5 6 7 11


array([[False,  True,  True, False, False, False,  True, False, False,
         True, False, False],
       [ True, False,  True, False, False, False, False,  True, False,
        False,  True, False],
       [ True,  True, False, False, False, False, False, False,  True,
        False, False,  True],
       [ True, False, False, False,  True,  True, False, False, False,
         True, False, False],
       [False,  True, False,  True, False,  True, False, False, False,
        False,  True, False],
       [False, False,  True,  True,  True, False, False, False, False,
        False, False,  True],
       [False, False, False,  True, False, False, False,  True,  True,
         True, False, False],
       [False, False, False, False,  True, False,  True, False,  True,
        False,  True, False],
       [False, False, False, False, False,  True,  True,  True, False,
        False, False,  True]])

In [123]:
result = linprog(row_weights, 
                 A_ub=-cover_mat.T, 
                 b_ub=np.full(n_cols, -1),
                 bounds=(0, 1))

In [149]:
def is_cover(decision_vars, cover_mat):
    return np.all(cover_mat.T @ decision_vars >= 1)

best_score = float('inf')
best_solution = np.ones(n_rows, dtype=bool)

for _ in range(10):
    iter_vars = np.zeros(n_rows, dtype=bool)

    while not is_cover(iter_vars, cover_mat):
        for i in range(n_rows):
            if np.random.rand() <= result.x[i]:
                iter_vars[i] = 1
                
                if iter_vars.sum() >= np.ceil(np.sum(result.x)):
                    if is_cover(iter_vars, cover_mat):
                        break
    
    iter_score = np.dot(iter_vars, row_weights)
    
    if iter_score < best_score:
        best_score = iter_score
        best_solution = iter_vars
        
print(best_score)
print(*np.nonzero(best_solution)[0]+1)

15
2 3 4 7 8 9
